In [13]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [14]:
device

device(type='cpu')

## 1. Данные

In [15]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


## Нейронная сеть

In [16]:
#[784, 200, 100, 10]
model = nn.Sequential(nn.Linear(784, 200), nn.Tanh(), 
                      #nn.Linear(200, 100), nn.Sigmoid(), 
                      nn.Linear(200, 10))

In [17]:
model

Sequential(
  (0): Linear(in_features=784, out_features=200, bias=True)
  (1): Tanh()
  (2): Linear(in_features=200, out_features=10, bias=True)
)

In [21]:
W = torch.zeros(10, 100)
with torch.no_grad():
    model[4].weight.data = W

IndexError: index 4 is out of range

In [ ]:
#[784, 200, 100, 10]
class MyNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super().__init__()
        self.input_size = input_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.num_classes = num_classes
        
        self.fc1 = nn.Linear(input_size, hidden_size1)#fully connected
        self.func1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)#fully connected
        
        self.alt_fc1 = nn.Linear(input_size, hidden_size2)#fully connected
        self.alt_func1 = nn.Tanh()

        self.func2 = nn.Tanh()
        self.fc3 = nn.Linear(hidden_size2, num_classes)#fully connected
        
    def forward(self, x):
        out = self.func1(self.fc1(x))
        out = self.func2(self.fc2(out))
        
        out2 = self.alt_func1(self.alt_fc1(x))
        
        out = out + out2
        out = self.fc3(out)
        
        return out

In [ ]:
model = MyNetwork(input_size=784, hidden_size1=200, hidden_size2=100, num_classes=10)

In [ ]:
model

In [ ]:
x, y = train_dataset[0]
model(x.flatten())

In [ ]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
#        self.fc2 = nn.Linear(hidden_size1, hidden_size2)  
#        self.fc3 = nn.Linear(hidden_size2, hidden_size3)  
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.relu(self.fc1(x))
#        out = self.relu(self.fc2(out))
#        out = self.relu(self.fc3(out))
        out = self.fc2(out)
        
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [ ]:
model

## 3. Лосс функция

In [ ]:
# Loss
criterion = nn.CrossEntropyLoss()

## 4. optimizer и обучение

In [ ]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
model = model.to(device)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
ss = []
for p in model.parameters():
    ss.append(p.numel())
print(ss)
sum(ss)